# 146. LRU Cache

---

## 📝 Description & Goal  
Design a data structure that supports

| Operation | Action | Time Complexity |
|-----------|--------|-----------------|
| `get(key)` | return value if the key exists, else `-1` | **O(1)** |
| `put(key, value)` | insert / update `(key,value)`; if capacity exceeded, evict **Least-Recently-Used** entry | **O(1)** |

---

## 🧩 Approach — Hash Map + Doubly Linked List  

| Component | Purpose | Why O(1) |
|-----------|---------|----------|
| **Hash Map** `key → node` | direct access to the cache node | dictionary lookup |
| **Doubly Linked List** `head ⇢ … ⇢ tail` | maintains usage order (MRU → LRU) | pointer tweaks only |

*Sentinel* nodes `head` (dummy MRU anchor) and `tail` (dummy LRU anchor) eliminate edge cases.

### Core helper methods
* ` _remove(node)` – unlink `node` from its current spot.  
* ` _insert_to_front(node)` – place `node` right after `head` (marks it MRU).  
* On every `get` / `put`, promote the accessed or newly inserted node with those two helpers.  
* When size > capacity, evict `tail.prev` (the real LRU node) and drop it from both the list and the map.

---

## 🔬 Linked-List Pointer Surgery

### `_remove(node)`

prev = node.prev
nxt  = node.next
prev.next = nxt   ← stitch forward
nxt.prev  = prev  ← stitch back



Before:  prev ⇆ node ⇆ nxt
After:   prev ⇆ nxt

### `_insert_to_front(node)`

first = head.next
node.prev = head
node.next = first
head.next = node
first.prev = node



Before:  head ⇆ first …
After:   head ⇆ node ⇆ first …

Only **four pointer writes** per insertion, **two** per removal—no traversal.

---

## 🧪 Example Walkthrough (`capacity = 2`)

| Step | Operation | List (`head ⇢ … ⇢ tail`) | Map Keys | Notes |
|------|-----------|---------------------------|----------|-------|
| 0 | init | `H ⇢ T` | ∅ | dummies linked |
| 1 | `put(1,1)` | `H (1) T` | {1} | insert 1 |
| 2 | `put(2,2)` | `H (2)(1) T` | {1,2} | insert 2 (MRU) |
| 3 | `get(1)` → 1 | `H (1)(2) T` | {1,2} | move 1 to front |
| 4 | `put(3,3)` | `H (3)(1) T` | {1,3} | over-cap → evict 2 |
| 5 | `get(2)` → -1 | *unchanged* | *unchanged* | 2 was evicted |
| 6 | `put(4,4)` | `H (4)(3) T` | {3,4} | evict 1 |
| 7 | `get(1)` → -1 | *unchanged* | *unchanged* | 1 gone |
| 8 | `get(3)` → 3 | `H (3)(4) T` | {3,4} | move 3 front |
| 9 | `get(4)` → 4 | `H (4)(3) T` | {3,4} | move 4 front |

---

### ⏱️ Complexities  
* **Time**: `get`, `put` → O(1)  
* **Space**: O(capacity) for the map + list nodes

In [ ]:
# Node for doubly linked list
class Node:
    def __init__(self, key: int, value: int):
        self.key = key        # key to identify entry in hashmap
        self.value = value    # actual value associated with key
        self.prev = None      # previous node in the list
        self.next = None      # next node in the list

# LRUCache class
class LRUCache:

    def __init__(self, capacity: int):
        self.cache = {}              # key -> Node mapping for O(1) access
        self.capacity = capacity     # max number of items allowed

        # Create dummy head and tail to avoid null checks on the ends
        self.head = Node(0, 0)
        self.tail = Node(0, 0)

        # Initially link head and tail
        self.head.next = self.tail
        self.tail.prev = self.head

    # Removes a node from its current position in the doubly linked list
    def _remove(self, node):
        prev = node.prev
        nxt = node.next
        prev.next = nxt      # skip over node in forward direction
        nxt.prev = prev      # skip over node in backward direction

    # Inserts a node right after head (most recently used position)
    def _insert_to_front(self, node): 
        first = self.head.next     # current first real node
        node.prev = self.head      # new node's prev points to dummy head
        node.next = first          # new node's next points to old first
        self.head.next = node      # head now points to new node
        first.prev = node          # old first's prev now points back to new node

    # Retrieves value by key and marks it as recently used
    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1              # not in cache

        node = self.cache[key]     # get node from hash map
        self._remove(node)         # remove from current position
        self._insert_to_front(node) # re-insert at front (MRU position)
        return node.value

    # Inserts or updates a key-value pair
    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            # If key exists, remove the old node from list
            self._remove(self.cache[key])

        # Create new node and put into hash map
        node = Node(key, value)
        self.cache[key] = node
        self._insert_to_front(node)  # add to front (most recently used)

        if len(self.cache) > self.capacity:
            # Evict least recently used (node before tail)
            lru = self.tail.prev
            self._remove(lru)         # remove from list
            del self.cache[lru.key]   # remove from hash map


# Usage:
# obj = LRUCache(capacity)
# val = obj.get(key)
# obj.put(key, value)

🔧 Doubly-Linked-List Surgery — Up Close & Personal

Think of the list as a conga line: every node holds both the hand in front (next) and the hand behind (prev).
Our job is to swap dancers without making anyone face-plant. 😎

H ⇆ A ⇆ B ⇆ C ⇆ T
^               ^
head           tail

We keep two dummy sentinels (head, tail) so there are never null edges to check. Life’s too short for if-else soup.

⸻

1️⃣ _remove(node)

def _remove(self, node):
    prev = node.prev       # dancer behind me
    nxt  = node.next       # dancer in front
    prev.next = nxt        # link them together...
    nxt.prev  = prev       # ...both directions
    # node is now isolated; its old .prev/.next are "dangling"

Step	Before	After
①	prev ⇆ node ⇆ nxt	prev ⇆ nxt
②	node.prev = prev (unchanged)	node.prev still points at prev (optional to null out)
③	node.next = nxt (unchanged)	node.next still points at nxt (safe to null)

Why it’s O(1): We already have the node reference from the hash-map, so no traversal.

Common rookie bug: forgetting to update both links; the line nxt.prev  = prev prevents the list splitting in half.

⸻

2️⃣ _insert_to_front(node)

def _insert_to_front(self, node):
    first = self.head.next    # current MRU node
    node.prev = self.head     # step 1: hook back-pointer
    node.next = first         # step 2: hook forward-pointer
    self.head.next = node     # step 3: head now points to node
    first.prev = node         # step 4: back-link from old first

Step	Before	After
–	H ⇆ first ⇆ … ⇆ T	H ⇆ node ⇆ first ⇆ … ⇆ T
🔑	The four assignments are exactly the hand-offs required to slot a new dancer behind the leader.	


⸻

3️⃣ get(key)

def get(self, key):
    if key not in self.cache:
        return -1

    node = self.cache[key]
    self._remove(node)        # yank from current spot
    self._insert_to_front(node)  # re-queue as MRU
    return node.value

The value comes free, because each node stores it.

⸻

4️⃣ put(key, value)

def put(self, key, value):
    if key in self.cache:     # updating old key
        self._remove(self.cache[key])

    node = Node(key, value)   # fresh or updated node
    self.cache[key] = node
    self._insert_to_front(node)

    if len(self.cache) > self.capacity:
        lru = self.tail.prev  # the wallflower before tail
        self._remove(lru)     # unlink from dance floor
        del self.cache[lru.key]

Eviction is one constant-time peek at tail.prev. No loops, no drama.

⸻

🖼️ Visual Trace (one insert)

Start: H ⇆ A ⇆ B ⇆ C ⇆ T

_put(4,4)  capacity = 3_

1) make Node(4)
2) insert_to_front(4):

   - first = H.next  -> A
   - node.prev = H
   - node.next = A
   - H.next = 4
   - A.prev = 4

   Result:
   H ⇆ 4 ⇆ A ⇆ B ⇆ C ⇆ T

3) size > cap → evict LRU:
   lru = T.prev -> C
   remove(C)
   T.prev = B ; B.next = T

   Final:
   H ⇆ 4 ⇆ A ⇆ B ⇆ T


⸻

🥡 Key take-aways
	1.	Four pointer writes per insertion, two per deletion — that’s it.
	2.	Dummies remove edge-case headaches at the ends.
	3.	Hash-map ensures we never search the list.
	4.	Capacity check is a single pointer hop; eviction shares the same O(1) primitives.

Let me know if you want to dig even deeper (e.g., memory footprint, thread-safety tweaks, or swapping the list for an OrderedDict on CPython). Otherwise—boom, you’ve mastered the pointer tango. 💃🕺